In [1]:
import sys, json, random, os
sys.path.append(sys.path.append(os.path.join(os.getcwd(), '..')))
from datamart.index_builder import IndexBuilder
from datamart.query_manager import QueryManager

In [2]:
es_index = "datamart"

qm = QueryManager()

Query any metadata has a column related temporal_coverage which has data in range `2018-01-01` to `2018-10-01`

In [3]:
query_1 = json.dumps({
    "query": {
        "bool": {
            "must": [
                {
                    "range": {
                        "variables.temporal_coverage.start": {
                            "lte": "2018-01-01T00:00:00"
                        }
                    }
                },
                {
                    "range": {
                        "variables.temporal_coverage.end": {
                            "gte": "2018-10-01T00:00:00"
                        }
                    }
                }
            ]
        }
    }
})


In [4]:
hitted_metadatas = qm.search(index=es_index, body=query_1)

print(len(hitted_metadatas))

48


In [6]:
for metadata in hitted_metadatas[10:20]:
    print("====== HIT a metadata on {}======".format(metadata["provenance"]))
    print("\n")
    print("====== GET the dataset ======")
    df = qm.get_dataset(metadata=metadata, variables=None, constrains={
        "locations": ["los angeles", "new york"],
        "date_range": {
            "start": "2018-09-23T00:00:00",
            "end": "2018-10-01T00:00:00"
        }
    })
    if df.shape[0] >= 10:
        print(df.iloc[random.sample(range(1, df.shape[0]), 10), :])
    else:
        print(df)
    print("\n\n")

====== HIT a metadata on noaa.org======


====== GET the dataset ======


                   date          stationid         city TOBS
35  2018-09-28T00:00:00  GHCND:USC00047326  los angeles  206
12  2018-09-24T00:00:00  GHCND:USW00003122  los angeles  200
21  2018-09-26T00:00:00  GHCND:USC00041194  los angeles  161
24  2018-09-26T00:00:00  GHCND:USC00049152  los angeles  217
29  2018-09-27T00:00:00  GHCND:USC00046719  los angeles  194
64  2018-09-24T00:00:00  GHCND:USC00280907     new york  172
22  2018-09-26T00:00:00  GHCND:USC00042214  los angeles  206
59  2018-10-01T00:00:00  GHCND:USW00003122  los angeles  244
84  2018-09-28T00:00:00  GHCND:USC00283704     new york  133
93  2018-09-30T00:00:00  GHCND:USC00283704     new york  117



====== HIT a metadata on tradingeconomics.com======


====== GET the dataset ======


             Country           Category                DateTime   Close  \
5576           Spain  5 Year Note Yield   8/29/2016 12:00:00 AM  0.0790   
7287          France  5 Year Note Yield   6/23/2017 12:00:00 AM -0.2030   
879           France  5 Year Note Yield   7/23/2014 12:00:00 AM  0.5470   
4389           Italy  5 Year Note Yield   2/11/2016 12:00:00 AM  0.6580   
6165   United States  5 Year Note Yield   12/8/2016 12:00:00 AM  1.8374   
8166         Germany  5 Year Note Yield  11/24/2017 12:00:00 AM -0.3250   
9077   United States  5 Year Note Yield    5/4/2018 12:00:00 AM  2.7837   
7409           Japan  5 Year Note Yield   7/14/2017 12:00:00 AM -0.0450   
3920       Australia  5 Year Note Yield  11/26/2015 12:00:00 AM  2.2870   
2942  United Kingdom  5 Year Note Yield   6/22/2015 12:00:00 AM  1.5400   

     Frequency HistoricalDataSymbol             LastUpdate  
5576     Daily            SPAIN5YNY   9/12/2016 6:09:00 PM  
7287     Daily           FRANCE5YNY   6/26/2017 3:19

            Country           Category                DateTime  Close  \
140   United States  4 Week Bill Yield   2/27/2007 12:00:00 AM  5.110   
2937  United States  4 Week Bill Yield   9/29/2017 12:00:00 AM  0.949   
1544  United States  4 Week Bill Yield   6/20/2012 12:00:00 AM  0.056   
900   United States  4 Week Bill Yield   1/12/2010 12:00:00 AM  0.008   
2993  United States  4 Week Bill Yield  12/18/2017 12:00:00 AM  1.238   
354   United States  4 Week Bill Yield  12/19/2007 12:00:00 AM  2.530   
533   United States  4 Week Bill Yield   8/25/2008 12:00:00 AM  1.620   
2732  United States  4 Week Bill Yield  12/16/2016 12:00:00 AM  0.459   
3067  United States  4 Week Bill Yield   3/30/2018 12:00:00 AM  1.621   
1604  United States  4 Week Bill Yield   9/11/2012 12:00:00 AM  0.091   

     Frequency HistoricalDataSymbol             LastUpdate  
140      Daily  UNITEDSTA4WEEBILYIE  11/14/2016 6:10:00 PM  
2937     Daily  UNITEDSTA4WEEBILYIE   10/2/2017 8:35:00 AM  
1544     Dail

            Country                   Category                DateTime  Close  \
608   United States  Natural Gas Stocks Change    9/2/2005 12:00:00 AM   36.0   
1234  United States  Natural Gas Stocks Change   8/25/2017 12:00:00 AM   30.0   
175   United States  Natural Gas Stocks Change   5/16/1997 12:00:00 AM   58.0   
1217  United States  Natural Gas Stocks Change   4/28/2017 12:00:00 AM   67.0   
724   United States  Natural Gas Stocks Change  11/23/2007 12:00:00 AM  -12.0   
76    United States  Natural Gas Stocks Change   6/23/1995 12:00:00 AM   87.0   
1072  United States  Natural Gas Stocks Change   7/25/2014 12:00:00 AM   88.0   
519   United States  Natural Gas Stocks Change  12/19/2003 12:00:00 AM -151.0   
350   United States  Natural Gas Stocks Change   9/22/2000 12:00:00 AM   74.0   
1168  United States  Natural Gas Stocks Change   5/20/2016 12:00:00 AM   71.0   

     Frequency  HistoricalDataSymbol            LastUpdate  
608     Weekly  UNITEDSTANATGASSTOCH  5/28/2015

...

            Country               Category                DateTime    Close  \
464   United States  Mortgage Applications   4/26/2013 12:00:00 AM      1.8   
420   United States  Mortgage Applications   8/10/2012 12:00:00 AM     -4.5   
85   United Kingdom  Mortgage Applications  10/31/2004 12:00:00 AM  58042.0   
391  United Kingdom  Mortgage Applications   2/29/2012 12:00:00 AM  33204.0   
738   United States  Mortgage Applications   8/18/2017 12:00:00 AM     -0.5   
670   United States  Mortgage Applications   7/22/2016 12:00:00 AM    -11.2   
35   United Kingdom  Mortgage Applications   8/31/2000 12:00:00 AM  52933.0   
454  United Kingdom  Mortgage Applications   2/28/2013 12:00:00 AM  31009.0   
376   United States  Mortgage Applications   12/2/2011 12:00:00 AM     12.8   
387   United States  Mortgage Applications    2/3/2012 12:00:00 AM      7.5   

    Frequency HistoricalDataSymbol             LastUpdate  
464    Weekly      UNITEDSTAMORAPP   5/28/2015 6:17:00 PM  
420    Wee

           Country                    Category                DateTime  \
74   United States  API Crude Oil Stock Change   11/1/2013 12:00:00 AM   
303  United States  API Crude Oil Stock Change   3/23/2018 12:00:00 AM   
299  United States  API Crude Oil Stock Change   2/23/2018 12:00:00 AM   
91   United States  API Crude Oil Stock Change   2/28/2014 12:00:00 AM   
8    United States  API Crude Oil Stock Change   5/18/2012 12:00:00 AM   
129  United States  API Crude Oil Stock Change  11/21/2014 12:00:00 AM   
177  United States  API Crude Oil Stock Change  10/23/2015 12:00:00 AM   
116  United States  API Crude Oil Stock Change   8/22/2014 12:00:00 AM   
244  United States  API Crude Oil Stock Change    2/3/2017 12:00:00 AM   
239  United States  API Crude Oil Stock Change  12/30/2016 12:00:00 AM   

      Close Frequency HistoricalDataSymbol             LastUpdate  
74    0.870    Weekly          APICRUDEOIL  11/28/2017 4:49:00 PM  
303   5.321    Weekly          APICRUDEOIL    4/3

             Country           Category                DateTime       Close  \
5844          Brazil  Consumer Spending  12/31/2007 12:00:00 AM  432651.751   
5413         Belgium  Consumer Spending   9/30/2006 12:00:00 AM   47540.000   
623   United Kingdom  Consumer Spending   3/31/1975 12:00:00 AM  111933.000   
6323           India  Consumer Spending   3/31/2009 12:00:00 AM    6709.200   
5323        Thailand  Consumer Spending   3/31/2006 12:00:00 AM  991603.000   
4731           Spain  Consumer Spending   6/30/2004 12:00:00 AM  122031.000   
2104     Switzerland  Consumer Spending   9/30/1993 12:00:00 AM   63893.700   
9802         Austria  Consumer Spending   6/30/2018 12:00:00 AM   40819.200   
4029           Malta  Consumer Spending   3/31/2002 12:00:00 AM  809354.000   
9643       Hong Kong  Consumer Spending  12/31/2017 12:00:00 AM  464128.000   

      Frequency HistoricalDataSymbol             LastUpdate  
5844  Quarterly         BRAZILCONSPE   12/1/2017 2:44:00 PM  
5413  

                   date          stationid         city WESD
18  2018-09-30T00:00:00  GHCND:US1NYWC0018     new york    0
8   2018-09-30T00:00:00  GHCND:US1CALA0001  los angeles    0
1   2018-09-24T00:00:00  GHCND:US1CALA0001  los angeles    0
4   2018-09-27T00:00:00  GHCND:US1CALA0001  los angeles    0
14  2018-09-26T00:00:00  GHCND:US1NYWC0018     new york    0
19  2018-10-01T00:00:00  GHCND:US1NJMD0062     new york    0
7   2018-09-29T00:00:00  GHCND:US1CALA0001  los angeles    0
20  2018-10-01T00:00:00  GHCND:US1NYWC0018     new york    0
12  2018-09-24T00:00:00  GHCND:US1NYWC0018     new york    0
13  2018-09-25T00:00:00  GHCND:US1NYWC0018     new york    0



====== HIT a metadata on tradingeconomics.com======


====== GET the dataset ======


         Country      Category                DateTime  Close Frequency  \
4468       Egypt  Lending Rate    4/9/2013 12:00:00 AM  10.75     Daily   
8678       Egypt  Lending Rate   4/14/2017 12:00:00 AM  15.75     Daily   
8566      Turkey  Lending Rate    3/3/2017 12:00:00 AM   9.25     Daily   
5244      Serbia  Lending Rate  12/31/2013 12:00:00 AM  12.00   Monthly   
2147   Mauritius  Lending Rate   1/31/2011 12:00:00 AM   8.50   Monthly   
1711  Mozambique  Lending Rate   8/31/2010 12:00:00 AM  16.89   Monthly   
4669      Turkey  Lending Rate   6/19/2013 12:00:00 AM   6.50     Daily   
9264       Egypt  Lending Rate   11/7/2017 12:00:00 AM  19.75     Daily   
5451   Euro Area  Lending Rate   3/13/2014 12:00:00 AM   0.75     Daily   
5093   Euro Area  Lending Rate  11/11/2013 12:00:00 AM   1.00     Daily   

     HistoricalDataSymbol             LastUpdate  
4468          EGYPTLENRAT  7/21/2015 12:38:00 PM  
8678          EGYPTLENRAT  4/15/2017 10:26:00 AM  
8566         TURKEYLE